In [26]:
from typing import Callable, Literal, Union
import numpy as np

CreditFormulas = dict[tuple[float | int,
                            float | int], Callable[[float], float]]
Floats = Union[float, np.ndarray]


class Tax():
    def __init__(self,
                 tax_percentages: tuple[float, ...] = (0.3697, 0.4950),
                 tax_thresholds: list[float] = [75518],
                 credit_labor_formulas: CreditFormulas = {
                     (0, 11491): lambda x: 0.08425 * x,
                     (11491, 24821): lambda x: 968 + 0.31433 * (x - 11490),
                     (24821, 39958): lambda x: 5158 + 0.02471 * (x - 24820),
                     (39958, 124935): lambda x: 5532 - 0.06510 * (x - 39958),
                     (124935, float('inf')): lambda x: 0.0},
                 credit_general_formulas: CreditFormulas = {
                     (0, 24813): lambda x: 3362,
                     (24813, 75518): lambda x: 3362 - 0.06630 * (x - 24812),
                     (75518, float('inf')): lambda x: 0.0
                 }
                 ) -> None:
        
        if not len(tax_percentages) - 1 == len(tax_thresholds):
            raise ValueError("tax_percentages is not long enough")

        self.tax_percentages: tuple[float, ...] = tax_percentages
        self.tax_thresholds: list[float] = tax_thresholds
        self.credit_labor_formulas: CreditFormulas = credit_labor_formulas
        self.credit_general_formulas: CreditFormulas = credit_general_formulas

    def calculate_gross_taxes(self, total_gross) -> float:
        percentages: tuple[float, ...] = self.tax_percentages
        thresholds: list[float] = self.tax_thresholds.copy()
        thresholds.append(thresholds[-1])

        sum_gross_taxes: float = 0

        for idx, (percent, threshold) in enumerate(
            iterable=zip(percentages, thresholds)
        ):
            if idx == 0:
                sum_gross_taxes += np.minimum(threshold, total_gross) * percent
            elif idx == len(percentages) - 1:
                sum_gross_taxes += np.maximum(total_gross -
                                              threshold, 0) * percent
            else:
                max_taxable_salary = np.maximum(
                    thresholds[idx + 1] - threshold, 0)
                sum_gross_taxes += np.minimum(max_taxable_salary,
                                              total_gross - threshold) * percent

        return sum_gross_taxes

    def calculate_credit(self, total_gross,
                         type: Literal['labor', 'general']) -> float:
        ranges_options: dict[str, CreditFormulas] = {
            'labor': self.credit_labor_formulas,
            'general': self.credit_general_formulas
        }

        salary_ranges = ranges_options[type]

        for (l, u), function in salary_ranges.items():
            if l <= total_gross < u:
                return function(total_gross)

        return 0.0

    def calculate_net_salary(self, total_gross) -> float:
        taxes_gross: float = self.calculate_gross_taxes(total_gross)
        labor_credit: float = self.calculate_credit(total_gross, type='labor')
        labor_general: float = self.calculate_credit(
            total_gross, type='general')
        return total_gross - (taxes_gross - (labor_credit + labor_general))


class Salary():
    def __init__(self, gross: Floats, holiday_pay: float = 8.0,
                 thirteenth: float = 100/12) -> None:
        self.gross: Floats = gross
        self.holiday_pay: float = holiday_pay
        self.thirteenth: float = thirteenth

    def calculate_gross_salary_yearly(self) -> Floats:
        rate_bonus: float = (100 + self.holiday_pay + self.thirteenth)/100
        return 12 * self.gross * rate_bonus

In [29]:
#| label: code-salary-example
# Instantiate a tax system and simulate a salary of 3000 gross with
# minimal holiday allowance (8%) and a so called 'thirteenth month'
tax = Tax()
salary = Salary(gross=3000)
gross_salary_year = salary.calculate_gross_salary_yearly()
print(f"Gross salary (yearly) = {gross_salary_year}")
print(f'Net salary (yearly) = {tax.calculate_net_salary(gross_salary_year)}')

Gross salary (yearly) = 41880.0
Net salary (yearly) = 34034.233400000005


In [34]:
gross_salaries = 1000* np.arange(start=2, stop=10, step=0.1)
gross_salaries_year = Salary(gross_salaries).calculate_gross_salary_yearly()
gross_taxes: float = tax.calculate_gross_taxes(gross_salaries_year)

import pandas as pd
df = pd.DataFrame({"gross_salary_monthly": gross_salaries, 
                   "gross_salary_yearly": gross_salaries_year, 
                   "gross_taxes": gross_taxes})
print(df)

    gross_salary_monthly  gross_salary_yearly  gross_taxes
0                 2000.0              27920.0   10322.0240
1                 2100.0              29316.0   10838.1252
2                 2200.0              30712.0   11354.2264
3                 2300.0              32108.0   11870.3276
4                 2400.0              33504.0   12386.4288
..                   ...                  ...          ...
75                9500.0             132620.0   56184.4946
76                9600.0             134016.0   56875.5146
77                9700.0             135412.0   57566.5346
78                9800.0             136808.0   58257.5546
79                9900.0             138204.0   58948.5746

[80 rows x 3 columns]
